<a href="https://colab.research.google.com/github/Yassmina-Abdo/realtime-head-pose-estimation/blob/main/Notebooks/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Liberaries

In [ ]:
import numpy as np
import pickle
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR,SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score

## Reading the prepared dataset and split it

In [ ]:
path_local='../Data/media_with_angles.csv'
path_drive='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/media_with_angles_low_normed.csv'
data =pd.read_csv(path_drive)
data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y463,y464,y465,y466,y467,y468,image_name,yaw,pitch,roll
0,-0.236824,-0.006193,-0.224011,-0.147134,0.032246,-0.006193,-0.159947,-1.133721,-0.159947,-0.121508,...,0.034117,-0.785904,-0.747465,-0.709027,-0.849968,-0.901219,image03195,0.725284,-0.140406,-0.174674
1,0.104239,-0.004250,0.056021,-0.076575,-0.016304,-0.016304,0.019859,-0.582855,0.007804,-0.004250,...,0.030482,-0.680720,-0.656611,-0.632502,-0.801262,-0.825371,image03247,-0.233745,-0.116700,0.095809
2,0.180066,-0.003883,0.180066,0.017758,-0.036345,-0.003883,0.115143,0.028579,0.115143,0.071861,...,0.047943,-0.525545,-0.482263,-0.460622,-0.633751,-0.698674,image03214,-1.330352,0.003072,0.362599
3,0.000114,-0.015663,0.031668,-0.110324,0.000114,0.000114,0.063222,-1.041159,0.078999,0.078999,...,-0.015938,-1.341195,-1.246533,-1.167649,-1.593625,-1.656732,image03238,-0.040907,-0.474087,0.019801
4,-0.107980,-0.005590,-0.056785,-0.069583,0.020008,0.032807,0.058404,-0.799111,0.084002,0.109599,...,0.043323,-0.852588,-0.814192,-0.775796,-0.916582,-0.954978,image03236,0.205732,-0.241359,-0.150232


In [ ]:
X = data.iloc[:,:-4]
X.head(1)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y459,y460,y461,y462,y463,y464,y465,y466,y467,y468
0,-0.236824,-0.006193,-0.224011,-0.147134,0.032246,-0.006193,-0.159947,-1.133721,-0.159947,-0.121508,...,0.008491,-0.029947,0.008491,0.021304,0.034117,-0.785904,-0.747465,-0.709027,-0.849968,-0.901219


In [ ]:
y = data.iloc[:,-3:]
y.head(1)

,yaw,pitch,roll
0,0.725284,-0.140406,-0.174674


## Scaling

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X.to_numpy() ,y, test_size=0.20, random_state=42, shuffle=True)

In [ ]:
x_train

array([[ 8.15231046e-02, -1.19403207e-02,  1.41159717e-03, ...,
        -6.62243832e-01, -9.69337944e-01, -1.03609753e+00],
       [ 2.90918909e-02, -5.37726414e-03, -2.83567008e-02, ...,
        -4.76083607e-01, -6.36939664e-01, -6.82898537e-01],
       [-1.28295671e-02, -1.84375331e-04, -1.84375331e-04, ...,
        -7.91349709e-01, -9.30446819e-01, -9.55737202e-01],
       ...,
       [-1.03836858e-01, -7.43262253e-03, -7.62927909e-02, ...,
        -8.71564959e-01, -1.10568953e+00, -1.14700563e+00],
       [ 9.80113504e-03, -9.71277054e-03,  4.88289462e-02, ...,
        -3.07637483e-01, -3.17394436e-01, -3.75936153e-01],
       [-1.85544390e-01, -3.03650827e-03, -1.24708429e-01, ...,
        -1.02310226e+00, -1.34756072e+00, -1.40839668e+00]])

In [ ]:
y_test

,yaw,pitch,roll
907,-0.443661,0.242653,-0.106793
135,-0.048311,0.045189,-0.461077
785,-0.001044,-0.167544,0.134020
1401,-0.079974,-0.342325,0.230787
598,0.096580,-0.249837,0.129833
...,...,...,...
100,-0.050280,-0.020617,0.044097
274,-0.427796,0.280815,0.319301
1210,0.312414,-0.143834,0.251424
101,0.809039,-0.077208,-0.160356


#### method for printing performane metrics

In [ ]:
def performance_metrics(y_true, y_pred):
  print(f"mean square error = {mean_squared_error(y_true, y_pred)}")
  print(f"mean absolute error = {mean_absolute_error(y_true, y_pred)}")
  print(f"R2-score = {r2_score(y_true, y_pred)}")

## **Model 1 MultiOutputRegressor:**

-  with RandomForestRegressor






In [ ]:
clf = MultiOutputRegressor(RandomForestRegressor(max_depth=12, random_state=42,n_estimators=300,min_samples_split=4))
clf.fit(x_train, y_train.to_numpy() )
clf.score(x_train, y_train.to_numpy() , sample_weight=None)

In [ ]:
y_pred= clf.predict(x_test)
clf.score(x_test, y_test.to_numpy(), sample_weight=None)

In [ ]:
performance_metrics(y_test.to_numpy(), y_pred)

In [ ]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_Rm.pkl', 'wb') as file:
        pickle.dump(clf, file)

## **Model 2 MultiOutputRegressor:**
  - SVR


In [ ]:
# define base model
model = SVR()
# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)
wrapper.fit(x_train, y_train.to_numpy())
y_pred = wrapper.predict(x_test)

In [ ]:
print(f"training score: {wrapper.score(x_train, y_train.to_numpy(), sample_weight=None)}")
print(f"testing score: {wrapper.score(x_test, y_test.to_numpy(), sample_weight=None)}")

training score: 0.6855620392925131
testing score: 0.7245691340366953


In [ ]:
performance_metrics(y_test.to_numpy(), y_pred)

mean square error = 0.03377016623168691
mean absolute error = 0.11022921509989768
R2-score = 0.7245691340366953


In [ ]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_svr.pkl', 'wb') as file:
        pickle.dump(wrapper, file)

## **Model 3 in MultiOutputRegressor:**

- GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42, n_estimators= 50, subsample=1) #random_state=42,n_estimators=100,learning_rate=0.05
model = MultiOutputRegressor(estimator=gbr)

In [ ]:
model.fit(x_train, y_train.to_numpy())
ypred = model.predict(x_test)

In [ ]:
print(f"training score: {model.score(x_train, y_train.to_numpy())}")
print(f"testing score: {model.score(x_test, y_test.to_numpy())}")

training score: 0.8791233329911495
testing score: 0.7135721941335916


In [ ]:
performance_metrics(y_test.to_numpy(), ypred)

mean square error = 0.03365530844960933
mean absolute error = 0.11387918416332442
R2-score = 0.7176061009828251


In [ ]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_GB.pkl', 'wb') as file:
        pickle.dump(model, file)

#**Single models trials**:

## yaw and roll model
- gradient boost regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42, n_estimators= 50, subsample=1) 
model = MultiOutputRegressor(estimator=gbr)
model.fit(x_train, y_train[['yaw']].to_numpy())

MultiOutputRegressor(estimator=GradientBoostingRegressor(n_estimators=50,
                                                         random_state=42,
                                                         subsample=1))

In [ ]:
print(f"training score: {model.score(x_train, y_train[['yaw']].to_numpy())}")
print(f"testing score: {model.score(x_test, y_test[['yaw']].to_numpy())}")

training score: 0.9225273908020336
testing score: 0.799039316376515


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42, n_estimators= 50, subsample=1) 
model_pitch = MultiOutputRegressor(estimator=gbr)
model_pitch.fit(x_train, y_train[['pitch']].to_numpy())

MultiOutputRegressor(estimator=GradientBoostingRegressor(n_estimators=50,
                                                         random_state=42,
                                                         subsample=1))

In [ ]:
print(f"training score: {model_pitch.score(x_train, y_train[['pitch']].to_numpy())}")
print(f"testing score: {model_pitch.score(x_test, y_test[['pitch']].to_numpy())}")

training score: 0.8440887140474439
testing score: 0.6184808564546651


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42, n_estimators= 50, subsample=1) 
model_roll = MultiOutputRegressor(estimator=gbr)
model_roll.fit(x_train, y_train[['roll']].to_numpy())

MultiOutputRegressor(estimator=GradientBoostingRegressor(n_estimators=50,
                                                         random_state=42,
                                                         subsample=1))

In [ ]:
print(f"training score: {model_roll.score(x_train, y_train[['roll']].to_numpy())}")
print(f"testing score: {model_roll.score(x_test, y_test[['roll']].to_numpy())}")

training score: 0.8707538941239709
testing score: 0.7231964095695946


In [ ]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_GB_yaw.pkl', 'wb') as file:
        pickle.dump(model, file)

In [ ]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_GB_pitch.pkl', 'wb') as file:
        pickle.dump(model_pitch, file)

In [ ]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_GB_roll.pkl', 'wb') as file:
        pickle.dump(model_roll, file)

## predict yaw only
- with gradient boost regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42, n_estimators= 50, subsample=1) 
gbr.fit(x_train, y_train['yaw'].to_numpy())

GradientBoostingRegressor(n_estimators=50, random_state=42, subsample=1)

In [ ]:
print(f"training score: {gbr.score(x_train, y_train['yaw'].to_numpy())}")
print(f"testing score: {gbr.score(x_test, y_test['yaw'].to_numpy())}")

training score: 0.9207339381456792
testing score: 0.8115012252305088


## predict pitch only
- with randomforest


In [ ]:
rf = RandomForestRegressor(random_state=42)#max_depth=12, random_state=42,n_estimators=300,min_samples_split=4
rf.fit(x_train, y_train['pitch'].to_numpy())

RandomForestRegressor(random_state=42)

In [ ]:
print(f"training score: {rf.score(x_train, y_train['pitch'].to_numpy())}")
print(f"testing score: {rf.score(x_test, y_test['pitch'].to_numpy())}")

training score: 0.9404554234466252
testing score: 0.626093730271344


- with svr

In [ ]:
svr = SVR()
svr.fit(x_train, y_train['pitch'].to_numpy())
print(f"training score: {svr.score(x_train, y_train['pitch'].to_numpy())}")
print(f"testing score: {svr.score(x_test, y_test['pitch'].to_numpy())}")


training score: 0.5531451225232049
testing score: 0.6030464890514908


In [ ]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_svr_pitch.pkl', 'wb') as file:
        pickle.dump(model, file)

## predict roll only 
- with greadient boosting regressor

In [ ]:
gbr.fit(x_train, y_train['roll'].to_numpy())

GradientBoostingRegressor(n_estimators=50, random_state=42, subsample=1)

In [ ]:
print(f"training score: {gbr.score(x_train, y_train['roll'].to_numpy())}")
print(f"testing score: {gbr.score(x_test, y_test['roll'].to_numpy())}")

training score: 0.8730395975594935
testing score: 0.7303402874327845
